# COGS 118A - Project Checkpoint

# Names

- Deepansha Singh
- Raina Song
- Ilya Kogan
- Winah Ruiz


# Abstract 
Natural language processing (NLP) has been making headlines all over the news in recent months due to the release and gaining popularity of ChatGPT (chatbot developed by OpenAI).
To take a closer look at the field of NLP, we plan to implement a sentiment analysis, a NLP technique that is used to determine the text attitude on a scale from -1 to 1 where absolute positive input has score 1, absolute negative input has score of -1, and a score of 0 is for neutral sentiment.
In this project we examine how general a sentiment analysis tool can be 
(e.g. when trained on data on a specific topic how well it can perform on other topics that it didn’t encounter before or general topicless data) 
by using different sampling techniques 
(k-fold Cross Validation and Nested Cross Validation) 
with different classifiers 
(Support Vector Classifier, k-Nearest Neighbors algorithm, Decision Trees, etc.) 
and quantifying their performance using various error metrics 
(precision, recall, or accuracy scores).
We will be using social media data from Reddit with sentiment labels to train our models, choose the best one according to an error metric, and then test how well our “best” model performs on a new dataset found on Twitter.


# Background

In previous studies, the generalisability and interoperability of NLP models in tasks such as fake news detection<a name="fakenews"></a>[<sup>[1]</sup>](#fakenews)<a name="fakenews2"></a>[<sup>[2]</sup>](#fakenews2), hate speech detection<a name="hatespeech"></a>[<sup>[3]</sup>](#hatespeech), and sentiment analysis<a name="sentian"></a>[<sup>[4]</sup>](#sentian) were explored. It was found that generalisability is a challenge for NLP algorithms, as they generalize poorly on new and unseen datasets<a name="fakenews"></a>[<sup>[1]</sup>](#fakenews). However, robust NLP models are desired to perform large-scale, cross-categorical classification tasks on the internet or social media platforms. Some studies suggest that the biases found in models may due to the small datasets the models are trained on, which make the models prone to overfitting issues<a name="fakenews"></a>[<sup>[1]</sup>](#fakenews). Larger and more diverse datasets are required to reduce biases in trained models. It was suggested that cross-dataset testing is a useful tool to evaluate model generalisability performance realistically<a name="hatespeech"></a>[<sup>[3]</sup>](#hatespeech). Due to such reasons, our project is motivated to explore the generalisability of NLP sentiment analysis models on different datasets pertaining to different subject matter.


# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

# Data


### (pavellexyr) The Reddit Climate Change Dataset: ###
Retrieved from: https://www.kaggle.com/datasets/pavellexyr/the-reddit-climate-change-dataset

There are two .csv files, one containing comments from Reddit on climate change and the other containing posts on climate change. For this project, we will only be using the .csv file with comments as there is already sentiment analysis done on the 4.6 million observations collected.

An observation in this dataset consists of:
* Type of datapoint (comment)
* Unique ID of the comment
* Unique ID of the comment’s subreddit
* The name of the subreddit the comment was found on
* If the comment’s subreddit is NSFW
* The timestamp (UTC) of the comment
* Permalink to the comment
* Body text of the comment
* Analyzed sentiment for the comment as a continuous value from [-1, 1]
* Comment’s score (votes on Reddit)

### (cosmos98) Twitter and Reddit Sentimental analysis Dataset: ###
Retrieved from: https://www.kaggle.com/datasets/cosmos98/twitter-and-reddit-sentimental-analysis-dataset

There are two .csv files, one containing comments from Reddit (36k observations) and the other containing tweets from Twitter (162k observations). The Twitter dataset was extracted with the focus on tweets people made about the Indian Prime Minister Narendra Modi. The Reddit dataset has no indication of having a specific area or topic they were sourced from. 

Both datasets only have two variables:
* The text of the comment or tweet
* The category/sentiment of the text {-1, 0, 1}

### (tirendazacademy) FIFA World Cup 2022 Tweets:  ###
Retrieved from: https://www.kaggle.com/datasets/tirendazacademy/fifa-world-cup-2022-tweets

There is one .csv file containing tweets regarding the 2022 FIFA World Cup, a dataset of about 22k observations.

An observation in this dataset contains:
* ID (index) of the observation
* Date the tweet was created
* Number of likes the tweet had
* Source of the tweet (Twitter of iPhone, Twitter for Android)
* The body text of the tweet
* Sentiment of the tweet as strings: “positive”, “neutral”, or “negative”


For all the datasets discussed above, there are only two variables we are concerned with: the text of the comment/tweet and the sentiment the text was already given. Due to some datasets being incredibly large, only the first 100k observations of each dataset will be used. 

All datasets will undergo further data cleaning. To the best of our ability, we will filter our dataset to only have English detected text using the langdetect library and regex. Other unusable observations (such as rows containing NaN values) will also be excluded. This results in slightly less than the upper limit of 100k observations initially taken from each dataset. In addition, the existing numeric labels for sentiment some datasets may have will be changed to string values of “positive”, “negative”, and “neutral” to be consistent with each other and only have 3 total classes for classification. 

The full code for cleaning the files used up to this point are in the “COGS118A replacement data cleaning.ipynb” file in this repository. Below will be code snippets, mainly of the functions created to clean the data. Demonstration will also be available in the other file as it is simply too large to include here. For demonstration purposes, these will just be example variable names instead of what was actually used in practice. 


In [4]:
"""
Libraries and global variables to be used for cleaning and limiting collected data to 100k observations at most. 
"""

import numpy as np
import pandas as pd

row_count = 1000
max_obv = 100

#https://pypi.org/project/langdetect/
### uncomment this to install, then comment and restart kernel ###
# %%capture
# !pip install langdetect

from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
### regex for more lang checking
import re

Taking the first 100k observations of a dataset and putting it into an initial DataFrame object.

In [ ]:
data_text = []
data_sentiment = []
i = 0
for chunk in pd.read_csv('dataset.csv', chunksize=row_count):
  if i < max_obv:
    data_text += chunk['text'].tolist()
    data_sentiment += chunk['sentiment'].tolist()
    i += 1
  else:
    break;

In [ ]:
df = pd.DataFrame(data={'text': data_text, 'sentiment': data_sentiment})
df = df.dropna()

Function to validate a text body. Valid text will be non-empty strings that are not solely whitespace of at least length 1. Regex and langdetect is used to keep observations that have Latin characters (so that it may be able to filter out text using solely Korean characters for example).

In [ ]:
def validate_line(line):
    if not line:
        return np.nan
    if line == "":
        return np.nan
    if not bool(line.strip()):
        return np.nan
    if len(line) < 1:
        return np.nan
    
    if bool(re.match('^(?=.*[a-zA-Z])', line)):
        try:
            if detect(line) != 'en':
                return np.nan
        except LangDetectException:
            return np.nan
    return True

Function to check if text body is English (detect returns 'en') for the entire dataset. 
* text_col takes a DataFrame.Series: df['text']
* sentiment_col takes a DataFrame.Series: df['sentiment']
Returns 3 lists of the same length, truncating the last chunk of observations that are less than 1000.

In [ ]:
def check_en(text_col, sentiment_col):
    en_text = text_col.tolist()
    en_sentiment = sentiment_col.tolist()
    lang = []
    
    start = 0
    for i in np.arange(row_count, len(en_text), row_count):
        #observations <1000 at the end will be lost but impact is negligible
        #!!!uncomment print statement below to show progress (recommended)!!!
#         print(start, i)
        lang += [validate_line(x) for x in en_text[start:i]]
        start = i
    print("Finished English check")
    ### all three return values should be of the same length
    return en_text[0:len(lang)], en_sentiment[0:len(lang)], lang

Putting the returned lists from check_en into a new DataFrame. df['english'] will be dropped after removing all rows with NaN values (non-English, non valid text bodies).

In [ ]:
en_data_text, en_data_sentiment, en_data_lang = check_en(df['text'], df['sentiment'])

en_df = pd.DataFrame(data={'text': en_data_text, 'sentiment': en_data_sentiment, 'english': en_data_lang})
en_df = en_df.dropna()
en_df = en_df.drop(columns=['english'])

Function to change numeric sentiment values into string values, otherwise it will just return the input value (if it was already string).

In [ ]:
def sentiment_to_string(sentiment):
    if type(sentiment) == int or type(sentiment) == float:
        if sentiment < 0:
            return "negative"
        if sentiment > 0:
            return "positive"
        return "neutral"
    else:
        return sentiment

Final clean dataset as a DataFrame after this line.

In [ ]:
en_df['sentiment'] = en_df['sentiment'].apply(sentiment_to_string)

# Proposed Solution

We have considered 3 different approaches for the proposed solution. For all three approaches, we aim to train different classifiers on a labelled dataset that we mentioned before, and test them on different labelled datasets to achieve the cross-dataset testing as mentioned in the Background section. This way, we can try to find out the generalisability of the models on different datasets and answer our research questions. Also, in each approach, different classifiers are evaluated and tested against our metrics and we will select the best model in each approach. For approaches 1 and 2, we are looking at supervised machine learning methods for the classifiers. And for approach 3, which is beyond the scope of the class (but if we have extra time) we will also do some exploration for some deep learning package’s accuracy on the dataset.
<br>

**Approach #1 : Supervised ML, using scikit packages to take care of BOTH the data preprocessing and also the NLP feature extraction.**

All the preprocessing will be taken care of by the CountVectorizer.
Please note that we are following this tutorial from the Scikit documentation <a name="scikit1"></a>[<sup>[9]</sup>](#scikit1). Since we haven’t previously worked with natural language processing problems in class before, we are  following this tutorial’s methods for feature extraction. Additionally, we did a lot of thorough, extensive research on various approaches for extracting features for text input. Some features <a name="scikit1"></a>[<sup>[9]</sup>](#scikit1)<a name="featextract"></a>[<sup>[10]</sup>](#featextract) we would be looking at are bag-of-words feature, tf-idf frequencies <a name="wiki"></a>[<sup>[11]</sup>](#wiki), and word embeddings <a name="mediium"></a>[<sup>[12]</sup>](#medium). 

After extracting all the various features, the next step would be to run the various classifiers <a name="scikit2"></a>[<sup>[13]</sup>](#scikit2) on the training dataset. Specifically, these are the following classifiers we are planning on running on these features as discussed in class: Support Vector Classifier, K-nearest neighbors, and Decision trees.
Some new classifiers, which we haven’t learned in class yet, which we are also planning to investigate in further detail are neural net, Naive Bayes, Ada Boost, etc. <a name="scikit2"></a>[<sup>[13]</sup>](#scikit2). We will be comparing these classifiers on different datasets and select the best model based on our metrics. 

In addition, for the training sampling techniques, we will be using the following to do comparisons between the different methods that we learned in class: K-fold validation with training, with training/validation/testing, and Nested cross validation. 
<br>

**Approach #2: Supervised ML, doing data preprocessing and above feature extraction from scratch.**

The steps for this approach will be similar to the first approach, however for the data preprocessing we will be doing everything from scratch instead of using CountVectorizer. 
Specifically, these are some of the data preprocessing techniques that we have researched about <a name="empstudy"></a>[<sup>[14]</sup>](#empstudy)<a name="featextractreview"></a>[<sup>[15]</sup>](#featextractreview) for natural language processing sub-field of machine learning are lemmatization/stemming, tokenization, and looking at part of speech for text.
Additionally, we will be looking at more research papers and trying to see more data preprocessing techniques for text input data.
We will again compare and contrast each classifier, and select the best model in this approach.
<br>

**Approach #3 (Above & Beyond, Extra work outside of class algorithms): Running on pre-trained neural networks.**

Along with supervised ML approaches, there has been a lot of extensive, thorough research being done in deep learning space for Natural Language Processing. Google’s BERT and Open AI’s GPT-3 <a name="top8"></a>[<sup>[16]</sup>](#top8) are just two of the popular NLP deep learning models out there. 
We will be loading these pretrained models in the Pytorch library. This is a library we haven’t learned in class, but is widely used in deep learning research problems. Specifically, this is new Machine learning content, in deep learning and transformers space, and we spent time understanding these really cool new ML concepts (links below). 
After performing the above tasks for the supervised machine learning component of the project, we will also be comparing the accuracy of this model to various pretrained models from HuggingFace for sentiment analysis, and see which model provides best accuracy, specifically BERT pretrained model <a name="huggingface1"></a>[<sup>[17]</sup>](#huggingface1) and Chat GPT-2 pretrained model <a name="huggingface2"></a>[<sup>[18]</sup>](#huggingface2).

If time permits, we will also try to train a neural network from scratch for sentiment analysis and do further research on research papers in this space. 

For all these different classification models, after we selected the best model by observing the results they produce on the labelled datasets, we will test them against an unlabelled dataset and see how accurate they seem on the unlabelled dataset compared to the labelled ones they were trained and tested on. In such a way, we can see not only the cross-dataset testing results on the generalisability of models we selected, and also the predictability of the models when they don't have labelled data.


# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

# Preliminary results

NEW SECTION!

Please show any preliminary results you have managed to obtain.

Examples would include:
- Analyzing the suitability of a dataset or alogrithm for prediction/solving your problem 
- Performing feature selection or hand-designing features from the raw data. Describe the features available/created and/or show the code for selection/creation
- Showing the performance of a base model/hyper-parameter setting.  Solve the task with one "default" algorithm and characterize the performance level of that base model.
- Learning curves or validation curves for a particular model
- Tables/graphs showing the performance of different models/hyper-parameters



# Ethics & Privacy

Due to the datasets all coming from real people collected through social media, there are ethical concerns over their consent in being part of this project as points of data, even with their words and personal information accessible through the media the data is extracted from (Twitter and Reddit).
Proper data cleaning and handling would require giving those people as much anonymity that is possible to be given, such as removing their usernames/handles and even the permalink/id to avoid being traced back to a user or community.

The goal of this project is to see if, given the labels already assigned in some datasets, how well can a model be trained on them to label other datasets?
As the data comes from a sample of people in the population of those respective media for certain topics, the text content may be biased towards/against certain topics or communities.
Some datasets have already included sentiment labels to indicate if they are positive, negative, or neutral.
Using this data in training may contribute to the model’s perceived connotation of those and related texts in unlabeled datasets.
Conclusions drawn from those results may be harmful to a degree regardless of actual intent or content because of the nature of applying sentiment analysis.
This project and those that are working with it do not show any personal support or opposition for any of the data gathered, and to keep the integrity of the project itself, all usable content (those in the English language) can be used without censorship as to not influence the direction of the model toward our personal preferences.


# Team Expectations 

* Communication channel is FB Messenger.
* Notify of any changes of plans in the communication channel as soon as possible.
* Confirm or reschedule meeting times for upcoming week if you know that you will not be able to meet at the specified time. 
* Come prepared to meetings with completed tasks.
* Ask for assistance anytime you feel like you need it.
* Don’t be afraid to pair up to learn some new skill together or help out one another

# Project Timeline Proposal

UPDATE THE PROPOSAL TIMELINE ACCORDING TO WHAT HAS ACTUALLY HAPPENED AND HOW IT HAS EFFECTED YOUR FUTURE PLANS

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM |  Brainstorm topics/questions (all)  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  10 AM |  Do background research on topic (Pelé) | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/1  | 10 AM  | Edit, finalize, and submit proposal; Search for datasets (Beckenbaur)  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/14  | 6 PM  | Import & Wrangle Data ,do some EDA (Maradonna) | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/23  | 12 PM  | Finalize wrangling/EDA; Begin programming for project (Cruyff) | Discuss/edit project code; Complete project |
| 3/13  | 12 PM  | Complete analysis; Draft results/conclusion/discussion (Carlos)| Discuss/edit full project |
| 3/19  | Before 11:59 PM  | NA | Turn in Final Project  |

# Footnotes
<a name="fakenews"></a>1.[^](#fakenews): LHoy, N., &amp; Koulouri, T. (2022). Exploring the generalisability of fake news detection models. *2022 IEEE International Conference on Big Data (Big Data)*. https://doi.org/10.1109/bigdata55660.2022.10020583<br> 
<a name="fakenews2"></a>2.[^](#fakenews2): Blackledge, C., &amp; Atapour-Abarghouei, A. (2021). Transforming fake news: Robust generalisable news classification using Transformers. *2021 IEEE International Conference on Big Data (Big Data)*. https://doi.org/10.1109/bigdata52589.2021.9671970<br>
<a name="hatespeech"></a>3.[^](#hatespeech): Yin W, Zubiaga A. 2021. Towards generalisable hate speech detection: a review on obstacles and solutions. *PeerJ Computer Science* 7:e598 https://doi.org/10.7717/peerj-cs.598<br>
<a name="sentian"></a>4.[^](#sentian): Moore, A., & Rayson, P. (2018). Bringing replication and reproduction together with generalisability in NLP: Three reproduction studies for target dependent sentiment analysis. *arXiv preprint arXiv:1806.05219*<br>
<a name="redditclimate"></a>5.[^](#redditclimate): https://www.kaggle.com/datasets/pavellexyr/the-reddit-climate-change-dataset<br>
<a name="twitterredditsentian"></a>6.[^](#twitterredditsentian): https://www.kaggle.com/datasets/cosmos98/twitter-and-reddit-sentimental-analysis-dataset<br>
<a name="twitterusairline"></a>7.[^](#twitterusairline): https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment<br>
<a name="fifatweets"></a>8.[^](#fifatweets): https://www.kaggle.com/datasets/tirendazacademy/fifa-world-cup-2022-tweets<br>
<a name="scikit1"></a>9.[^](#scikit1): https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#exercise-2-sentiment-analysis-on-movie-reviews<br>
<a name="featextract"></a>10.[^](#featextract): Zareapoor, M., &amp; K. R, S. (2015). Feature extraction or feature selection for text classification: A case study on phishing email detection. *International Journal of Information Engineering and Electronic Business*, 7(2), 60–65. https://doi.org/10.5815/ijieeb.2015.02.08<br>
<a name="wiki"></a>11.[^](#wiki): https://en.wikipedia.org/wiki/Tf%E2%80%93idf<br>
<a name="medium"></a>12.[^](#medium): https://medium.com/geekculture/text-feature-extraction-3-3-word-embeddings-model-e98f3d270dce<br>
<a name="scikit2"></a>13.[^](#scikit2): https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html<br>
<a name="empstudy"></a>14.[^](#empstudy): Sun, X., Liu, X., Hu, J., &amp; Zhu, J. (2014). Empirical studies on the NLP techniques for source code data preprocessing. *Proceedings of the 2014 3rd International Workshop on Evidential Assessment of Software Technologies*. https://doi.org/10.1145/2627508.2627514<br>
<a name="featextractreview"></a>15.[^](#featextractreview): Asghar, M. Z., Khan, A., Ahmad, S., & Kundi, F. M. (2014). A review of feature extraction in sentiment analysis. *Journal of Basic and Applied Scientific Research*, 4(3), 181-186.<br>
<a name="top8"></a>16.[^](#top8): https://analyticsindiamag.com/top-8-pre-trained-nlp-models-developers-must-know/<br>
<a name="huggingface1"></a>17.[^](#huggingface1): https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis?text=Yay%21%21<br>
<a name="huggingface2"></a>18.[^](#huggingface2): https://huggingface.co/michelecafagna26/gpt2-medium-finetuned-sst2-sentiment?text=yayy%21%21<br>